# **Studi kasus two way**

Pengaruh Jenis Pelatihan dan Tingkat Pendidikan terhadap Skor Kompetensi Karyawan

Deskripsi:
Sebuah perusahaan ingin mengevaluasi dua hal:
- Apakah jenis pelatihan (A atau B) berpengaruh terhadap hasil tes karyawan?
- Apakah tingkat pendidikan (SMA, S1, S2) juga berpengaruh?
- Apakah ada interaksi antara jenis pelatihan dan tingkat pendidikan?


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Data simulasi
data = pd.DataFrame({
    "Skor": [75, 78, 80, 85, 83, 88, 70, 72, 73, 76, 74, 77],
    "Pelatihan": ["A"] * 6 + ["B"] * 6,
    "Pendidikan": ["SMA", "S1", "S2"] * 4
})

# Visualisasi awal
plt.figure(figsize=(8,6))
sns.boxplot(x="Pendidikan", y="Skor", hue="Pelatihan", data=data)
plt.title("Distribusi Skor berdasarkan Pendidikan dan Pelatihan")
plt.show()


In [ ]:
from scipy.stats import shapiro

# Per grup kombinasi
for edu in data["Pendidikan"].unique():
    for pel in data["Pelatihan"].unique():
        subset = data[(data["Pendidikan"] == edu) & (data["Pelatihan"] == pel)]["Skor"]
        stat, p = shapiro(subset)
        print(f"Normalitas - {pel} & {edu}: p-value = {p:.4f}")


In [ ]:
from scipy.stats import levene

grouped = data.groupby(['Pendidikan', 'Pelatihan'])['Skor'].apply(list).tolist()
stat, p = levene(*grouped)
print(f"Levene Test (Homogenitas): p-value = {p:.4f}")


In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Model ANOVA 2 arah dengan interaksi
model = ols("Skor ~ C(Pelatihan) * C(Pendidikan)", data=data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)


In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Hanya untuk salah satu faktor (misalnya Pendidikan)
tukey = pairwise_tukeyhsd(endog=data['Skor'], groups=data['Pendidikan'], alpha=0.05)
print(tukey.summary())
